---
# library
---

In [1]:
import json
import requests
import pandas as pd
from datetime import datetime
import time

---
# code
---

In [ ]:
def get_public_holidays(start_year, end_year, country_code):
    url = "https://date.nager.at/api/v3/publicholidays/"
    yearly_data = {}

    for year in range(start_year, end_year + 1):
        api_url = f"{url}{year}/{country_code}"

        try:
            response = requests.get(api_url)
            if response.status_code == 200:
                data = response.json()
                yearly_data[year] = data
            else:
                print(f"Failed to retrieve data for {year}, status code: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")

        time.sleep(1)  # Para evitar sobrecargar el servidor con solicitudes

    return yearly_data


def generar_calendario(anio_inicio, anio_fin):
    fecha_inicio = datetime(anio_inicio, 1, 1)
    fecha_fin = datetime(anio_fin, 12, 31)
    fechas = pd.date_range(start=fecha_inicio, end=fecha_fin)

    calendario_df = pd.DataFrame({
        'fecha': fechas,
        'año': fechas.year,
        'mes': fechas.month,
        'semana': fechas.isocalendar().week,
        'dia_de_la_semana': fechas.dayofweek + 1  # 1 = Lunes, 7 = Domingo
    })

    calendario_df.reset_index(drop=True, inplace=True)

    return calendario_df

In [ ]:
start_year = 2024
end_year = start_year + 5
country_code = "CO"

public_holidays_data = get_public_holidays(start_year, end_year, country_code)
calendario = generar_calendario(start_year, end_year)
festivos_list = [
    {'fecha': festivo['date'], 'nombre_del_festivo': festivo['localName']}
    for anio, festivos in public_holidays_data.items()
    for festivo in festivos
]
festivos_df = pd.DataFrame(festivos_list)
festivos_df['fecha'] = pd.to_datetime(festivos_df['fecha'])
calendario = calendario.merge(festivos_df, on='fecha', how='left')
calendario['festivo'] = calendario['nombre_del_festivo'].apply(lambda x: 'Sí' if pd.notna(x) else 'No')
calendario['dia_habil'] = calendario.apply(
    lambda row: 'Sí' if row['dia_de_la_semana'] <= 5 and row['festivo'] == 'No' else 'No',
    axis=1
)


In [5]:
calendario.head()

,fecha,año,mes,semana,dia_de_la_semana,nombre_del_festivo,festivo,dia_habil
0,2024-01-01,2024,1,1,1,Año Nuevo,Sí,No
1,2024-01-02,2024,1,1,2,NaN,No,Sí
2,2024-01-03,2024,1,1,3,NaN,No,Sí
3,2024-01-04,2024,1,1,4,NaN,No,Sí
4,2024-01-05,2024,1,1,5,NaN,No,Sí


In [6]:
calendario.to_csv("calendario.csv", index=False, encoding="utf-8")